In [1]:
import sys
sys.path.append("..")

import torch
import torch.optim as optim

import matplotlib.pyplot as plt

from datasets import *
from ad import *

torch.manual_seed(1234)

/home/antonxue/lib/miniconda3/envs/arpro/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = EfficientAdModel(device="cpu")
state_dict = torch.load("../_dump/ad_eff_mvtec_bottle_best.pt", map_location="cpu")["model_state_dict"]
model.load_state_dict(state_dict)
model.eval().cpu()
;
torch.manual_seed(1236)
dataloader = get_ad_dataloader("mvtec", "vae", batch_size=8, category="bottle", split="test", normalize_image=False)
for batch in dataloader:
    break
print(batch["label"])
with torch.no_grad():
    x = batch["image"]
    out = model(2*x-1)
score = out.score
thresh = out.alpha.flatten(1).quantile(0.80, dim=1)
# batch["label"], score, thresh
print(score)
plt.clf()

fig, ax = plt.subplots(4,8, figsize=(12, 8))
for i in range(8):
    xi = x[i].detach()
    xhati = (out.others["map_ae"][i].detach()*0.5 + 0.5).clamp(0,1)
    alphai = out.alpha[i].max(dim=0).values.detach()
    alphai_thresh = alphai > thresh[i]
    ax[0,i].imshow(xi.numpy().transpose(1,2,0))
    ax[1,i].imshow(xhati.cpu().numpy().transpose(1,2,0))
    ax[2,i].imshow(alphai_thresh.cpu().numpy())
    ax[3,i].imshow(batch["mask"][i].numpy().transpose(1,2,0))


TypeError: EfficientAdModel.__init__() got an unexpected keyword argument 'device'

## fastflow

In [ ]:
cat = "bottle"
model = FastflowAdModel()
state_dict = torch.load(f"../_dump/ad_fast_mvtec_{cat}_best.pt")["model_state_dict"]
model.load_state_dict(state_dict)
model.eval()
;
torch.manual_seed(1236)
dataloader = get_ad_dataloader("mvtec", "vae", batch_size=8, category=cat, split="test", normalize_image=False)
for batch in dataloader:
    break
print(batch["label"])
with torch.no_grad():
    x = batch["image"]
    out = model(2*x-1)
score = out.score
thresh = out.alpha.flatten(1).quantile(0.95, dim=1)
# batch["label"], score, thresh
print(score)
plt.clf()
xs = []
for i in range(8):
    xi = x[i].detach()
    with torch.no_grad():
        score = model(2*xi.unsqueeze(0) - 1).score
        print(score.item())
        xs
fig, ax = plt.subplots(3,8, figsize=(12, 8))
for i in range(8):
    xi = x[i].detach()
    # xhati = (out.others["map_ae"][i].detach()*0.5 + 0.5).clamp(0,1)
    alphai = out.alpha[i].max(dim=0).values.detach()
    alphai_big = alphai > thresh[i]
    ax[0,i].imshow(xi.numpy().transpose(1,2,0))
    # ax[1,i].imshow(xhati.cpu().numpy().transpose(1,2,0))
    # ax[1,i].imshow(alphai.cpu().numpy())
    ax[2,i].imshow(alphai_big.cpu().numpy())


In [ ]:
cat = "transistor"
model = FastflowAdModel()
state_dict = torch.load(f"../_dump/ad_fast_mvtec_{cat}_best.pt")["model_state_dict"]
model.load_state_dict(state_dict)
model.eval()
;
torch.manual_seed(1236)
dataloader = get_ad_dataloader("mvtec", "vae", batch_size=8, category=cat, split="test", normalize_image=False)
for batch in dataloader:
    break
print(batch["label"])
with torch.no_grad():
    x = batch["image"]
    out = model(2*x-1)
score = out.score
thresh = out.alpha.flatten(1).quantile(0.95, dim=1)
# batch["label"], score, thresh
print(score)
plt.clf()

fig, ax = plt.subplots(3,8, figsize=(12, 8))
for i in range(8):
    xi = x[i].detach()
    # xhati = (out.others["map_ae"][i].detach()*0.5 + 0.5).clamp(0,1)
    alphai = out.alpha[i].max(dim=0).values.detach()
    alphai_big = alphai > thresh[i]
    ax[0,i].imshow(xi.numpy().transpose(1,2,0))
    # ax[1,i].imshow(xhati.cpu().numpy().transpose(1,2,0))
    ax[1,i].imshow(alphai.cpu().numpy())
    ax[2,i].imshow(alphai_big.cpu().numpy())


In [ ]:
model = EfficientAdModel()

In [ ]:
# model = VaeADModel()
model = EfficientAdModel()
state_dict = torch.load("../_dump/ad_eff_mvtec_transistor_last.pt")["model_state_dict"]

# state_dict = torch.load("../_dump/ad_vae_mvtec_transistor_rs1.00_ks2.00_cs0.1_best.pt")["model_state_dict"]
model.load_state_dict(state_dict)
model.eval()
;

In [ ]:
torch.manual_seed(1236)
dataloader = get_ad_dataloader("mvtec", "vae", batch_size=8, category="transistor", split="test", normalize_image=False)
for batch in dataloader:
    break

In [ ]:
batch["label"]

In [ ]:
with torch.no_grad():
    x = batch["image"]
    out = model(x)

In [ ]:
score = out.score
thresh = out.alpha.flatten(1).quantile(0.90, dim=1)
batch["label"], score, thresh

In [ ]:
plt.clf()

fig, ax = plt.subplots(4,8, figsize=(12, 8))
for i in range(8):
    xi = x[i].detach()
    xhati = (out.others["map_ae"][i].detach()*0.5 + 0.5).clamp(0,1)
    alphai = out.alpha[i].max(dim=0).values.detach()
    alphai_big = alphai > thresh[i]
    ax[0,i].imshow(xi.numpy().transpose(1,2,0))
    ax[1,i].imshow(xhati.cpu().numpy().transpose(1,2,0))
    ax[2,i].imshow(alphai.cpu().numpy())
    ax[3,i].imshow(alphai_big.cpu().numpy())
    

In [ ]:
out.others["x_recon"].shape